In [1]:
import pandas as pd
import datetime
import numpy as np
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
demand_df = pd.read_csv(r'hourly_demand_all_regions.csv', encoding='latin-1')
demand_df = pd.DataFrame(demand_df)
demand_df.rename({'Año':'Year', 'Mes':'Month', 'Día':'Day', 'Hora':'Hour'}, inplace=True, axis=1)
demand_df.drop(['La Paz', 'SIN', 'SIN8', 'SIN8+LPZ'], axis=1, inplace=True)
demand_df['09-Baja California Sur'] = demand_df['09-Baja California Sur'] - demand_df['10-Mulegé']
#demand_df = pd.melt(demand_df, id_vars= ['Year', 'Month','Day','Hour'], value_vars=[x for x in demand_df.columns if x not in ['Year', 'Month','Day','Hour']])
demand_df.rename({'variable':'Region'}, inplace=True, axis=1)
#demand_df.drop(['Day','Hour'], axis=1, inplace=True)

# Create column for weekday/weekend
demand_df['Date'] = demand_df['Year'].astype(str) + '-' + demand_df['Month'].astype(str) + '-' + demand_df['Day'].astype(str)
demand_df['Date'] = pd.to_datetime(demand_df['Date'], errors='coerce', format='%Y-%m-%d')
demand_df['Day-of-week'] = demand_df['Date'].dt.dayofweek
demand_df.drop('Date', axis=1, inplace=True)
demand_df['Day-of-week'][demand_df['Day-of-week']<5] = 'WD'
demand_df['Day-of-week'][demand_df['Day-of-week']!='WD'] = 'WE'


print(demand_df['Day-of-week'].unique())
print(demand_df['Year'].unique())

['WD' 'WE']
[2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031
 2032]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [3]:
seasons = {
    1:'SEA1',
    2:'SEA1',
    3:'SEA2',
    4:'SEA2',
    5:'SEA2',
    6:'SEA3',
    7:'SEA3',
    8:'SEA3',
    9:'SEA4',
    10:'SEA4',
    11:'SEA4',
    12:'SEA1',
}

day_parts = {
    'DP1':[0,7],
    'DP2':[7,14],
    'DP3':[14,21],
    'DP4':[21,24],
    
}

timeslices = {}


regions = [x for x in demand_df.columns if x not in ['Hour', 'Day', 'Month', 'Year', 'Day-of-week']]
years = demand_df['Year'].unique()


demand_df_hours = pd.DataFrame(columns=demand_df.columns)

for each_year in years:
    demand_df_year = demand_df[demand_df['Year']==each_year]
    demand_df_year.insert(4,'Hour_continuous',range(1,1+len(demand_df_year)))
    demand_df_hours = demand_df_hours.append(demand_df_year)

demand_df_hours['Month'].replace(seasons, inplace=True)
demand_df_hours['Total']  = demand_df_hours[regions].sum(axis=1)

df_yearsplit = demand_df_hours[['Total', 'Month', 'Day-of-week', 'Hour', 'Year']]
df_yearsplit = df_yearsplit[df_yearsplit['Year']==2018]

for each_season in df_yearsplit['Month'].unique():
    for each_day in df_yearsplit['Day-of-week'].unique():
        df_temp = df_yearsplit[(df_yearsplit['Month']==each_season) 
                           & (df_yearsplit['Day-of-week'] == each_day)].sort_values(by='Total')
        df_temp = df_temp.groupby(['Month', 'Day-of-week', 'Hour', 'Year']).mean().sort_values(by='Total')
        
        for each_DP in day_parts:
            #print(each_DP, day_parts[each_DP][0], day_parts[each_DP][1])
            df_temp_2 = df_temp[day_parts[each_DP][0]:day_parts[each_DP][1]]
            df_temp_2['Day_part'] = each_DP
            df_temp_2 = df_temp_2.reset_index()
            df_temp_2['Timeslice_key'] = df_temp_2['Month'] + df_temp_2['Day-of-week'] + df_temp_2['Hour'].astype(str)
            df_temp_2['Timeslice_value'] = df_temp_2['Month'] + df_temp_2['Day-of-week'] + df_temp_2['Day_part']
            #print(df_temp_2)
            timeslice_dict = dict(zip(df_temp_2['Timeslice_key'], 
                                      df_temp_2['Timeslice_value']))
            timeslices.update(timeslice_dict)

demand_df_hours['Timeslice'] = demand_df_hours['Month'] + demand_df_hours['Day-of-week'] + demand_df_hours['Hour'].astype(str)
#print(demand_df_hours)

demand_df_hours['Timeslice'].replace(timeslices, inplace=True)
print(demand_df_hours)


#demand_df = demand_df.groupby('Year').sum().mul(3.6*1e-6)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


          01-Central  02-Oriental  03-Occidental  04-Noroeste     05-Norte  \
0        5311.340372  4755.192227    5629.822779  1761.141904  1994.485230   
1        5040.856806  4565.008531    5418.374748  1695.095773  1936.889843   
2        4772.852624  4352.283610    5192.882069  1674.842196  1883.726974   
3        4481.027233  4155.010646    4984.024709  1643.794791  1826.469598   
4        4278.064801  4041.099882    4895.250512  1602.648095  1792.811618   
...              ...          ...            ...          ...          ...   
131491  10092.462140  8954.440631   11375.293350  3567.373102  3808.620594   
131492   9638.175644  8701.864739   10800.139880  3494.200208  3680.832861   
131493   8975.608659  8349.919893   10174.665320  3342.258458  3578.413311   
131494   8554.624246  7922.925893    9700.237951  3141.877733  3428.583306   
131495   8137.051363  7472.499345    9277.698011  2942.771372  3302.456978   

         06-Noreste  07-Peninsular  08-Baja California  \
0    

In [59]:
## YearSplit calculation

demand_df_yearsplit = demand_df_hours.drop(['Total', 'Month', 'Day', 'Day-of-week', 'Hour', 'Hour_continuous'] 
                                           +regions, 
                                           axis=1)
demand_df_yearsplit = demand_df_yearsplit.groupby('Year')['Timeslice'].value_counts(normalize=True).to_frame('value')
demand_df_yearsplit = demand_df_yearsplit.reset_index()
demand_df_yearsplit = pd.pivot_table(demand_df_yearsplit, 
                                   index='Timeslice', 
                                   values='value', 
                                   columns='Year', 
                                   aggfunc=np.sum)

#print(demand_df_yearsplit)

Year           2018      2019      2020      2021      2022      2023  \
Timeslice                                                               
SEA1WDDP1  0.051142  0.051941  0.052596  0.051142  0.050342  0.050342   
SEA1WDDP2  0.051142  0.051941  0.052596  0.051142  0.050342  0.050342   
SEA1WDDP3  0.051142  0.051941  0.052596  0.051142  0.050342  0.050342   
SEA1WDDP4  0.021918  0.022260  0.022541  0.021918  0.021575  0.021575   
SEA1WEDP1  0.020776  0.019977  0.019923  0.020776  0.021575  0.021575   
SEA1WEDP2  0.020776  0.019977  0.019923  0.020776  0.021575  0.021575   
SEA1WEDP3  0.020776  0.019977  0.019923  0.020776  0.021575  0.021575   
SEA1WEDP4  0.008904  0.008562  0.008538  0.008904  0.009247  0.009247   
SEA2WDDP1  0.052740  0.052740  0.051799  0.052740  0.052740  0.052740   
SEA2WDDP2  0.052740  0.052740  0.051799  0.052740  0.052740  0.052740   
SEA2WDDP3  0.052740  0.052740  0.051799  0.052740  0.052740  0.052740   
SEA2WDDP4  0.022603  0.022603  0.022199  0.022603  

In [19]:
## SpecifiedDemandProfile calculation

demand_df_regions = demand_df_hours.drop(['Month', 'Day', 'Day-of-week', 'Hour', 'Hour_continuous'], axis=1)

demand_df_regions = demand_df_regions.groupby(['Year', 'Timeslice']).sum()
demand_df_total = demand_df_regions.groupby(['Year']).sum()

demand_df_regions[regions] = demand_df_regions[regions]/demand_df_total[regions]
demand_df_regions = demand_df_regions.reset_index()

demand_df_regions = pd.melt(demand_df_regions, 
                            id_vars=['Year', 'Timeslice'], 
                            value_vars=regions)


demand_df_regions = pd.pivot_table(demand_df_regions, 
                                   index=['variable','Timeslice'], 
                                   values='value', 
                                   columns='Year', 
                                   aggfunc=np.sum)

demand_df_total = demand_df_total.mul(3.6*1e-6)
demand_df_total = demand_df_total.reset_index()
demand_df_total = pd.melt(demand_df_total, 
                            id_vars='Year', 
                            value_vars=regions)
demand_df_total = pd.pivot_table(demand_df_total, 
                                   index='variable', 
                                   values='value', 
                                   columns='Year', 
                                   aggfunc=np.sum)
#print(demand_df_regions)

Year                      2018      2019      2020      2021      2022  \
variable   Timeslice                                                     
01-Central SEA1WDDP1  0.041907  0.042364  0.043009  0.043017  0.043078   
           SEA1WDDP2  0.050493  0.050565  0.051464  0.050720  0.051860   
           SEA1WDDP3  0.053362  0.053302  0.054081  0.053179  0.054441   
           SEA1WDDP4  0.025034  0.025070  0.025336  0.024722  0.024847   
           SEA1WEDP1  0.017907  0.017607  0.017334  0.016961  0.016608   
...                        ...       ...       ...       ...       ...   
10-Mulegé  SEA4WDDP4  0.025987  0.025905  0.025732  0.025797  0.026355   
           SEA4WEDP1  0.020327  0.020565  0.020195  0.019659  0.019891   
           SEA4WEDP2  0.022854  0.023025  0.023093  0.022936  0.022382   
           SEA4WEDP3  0.024994  0.024821  0.024865  0.024906  0.023624   
           SEA4WEDP4  0.010417  0.010509  0.010642  0.010635  0.010085   

Year                      2023      2

In [60]:
demand_df_yearsplit.to_csv(r'yearsplit.csv')
demand_df_regions.to_csv(r'specified_demand_profiles.csv')
demand_df_total.to_csv(r'total_demand_PJ.csv')

In [38]:
demand_df_pvcf = demand_df_hours.drop(regions +
                                     ['Month', 'Day', 'Hour', 'Day-of-week', 'Total'],
                                     axis=1)
demand_df_pvcf = demand_df_pvcf[demand_df_pvcf['Year']==2018].drop('Year', axis=1)

df_pvcf = pd.read_csv(r'hourly_pvdata.csv')

demand_df_pvcf = pd.merge(demand_df_pvcf, df_pvcf, on=['Hour_continuous']).drop('Hour_continuous', axis=1)
demand_df_pvcf = demand_df_pvcf.groupby(['Timeslice']).mean()
print(demand_df_pvcf)

demand_df_pvcf.to_csv(r'pv_cf_timeslices.csv')

           capacity_factor
Timeslice                 
SEA1WDDP1         0.000069
SEA1WDDP2         0.349408
SEA1WDDP3         0.457002
SEA1WDDP4         0.000000
SEA1WEDP1         0.015643
SEA1WEDP2         0.203297
SEA1WEDP3         0.616060
SEA1WEDP4         0.000077
SEA2WDDP1         0.020831
SEA2WDDP2         0.405868
SEA2WDDP3         0.267775
SEA2WDDP4         0.101960
SEA2WEDP1         0.020725
SEA2WEDP2         0.310363
SEA2WEDP3         0.299582
SEA2WEDP4         0.217603
SEA3WDDP1         0.062937
SEA3WDDP2         0.319909
SEA3WDDP3         0.212494
SEA3WDDP4         0.072162
SEA3WEDP1         0.060637
SEA3WEDP2         0.230643
SEA3WEDP3         0.249060
SEA3WEDP4         0.184603
SEA4WDDP1         0.011730
SEA4WDDP2         0.388989
SEA4WDDP3         0.267108
SEA4WDDP4         0.000000
SEA4WEDP1         0.012016
SEA4WEDP2         0.311973
SEA4WEDP3         0.380626
SEA4WEDP4         0.000000
